In [ ]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive')
#drive.mount('/content/drive', force_remount=True)

In [1]:
!pip install transformers
!pip install sentencepiece
!pip install tensorflow==2.7.0
!pip install stanza
!pip install transformers
!pip install tensorflow-addons
!pip install nltk
!pip install textacy

In [1]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
from textacy.datasets.supreme_court import SupremeCourt
import numpy as np
import re
import unicodedata
import nltk
#from transformers import pipeline
from nltk.corpus import stopwords
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout, Input, BatchNormalization
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
import itertools
from sklearn.utils import shuffle
from tensorflow.keras import regularizers
#from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
import pandas as pd
from transformers import AutoTokenizer, TFAutoModel
import numpy as np
import gc
import math
import json
import stanza
from tensorflow.keras import *
import tensorflow as tf
from tensorflow.keras import *
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from transformers import TFRobertaModel,RobertaTokenizer,LongformerTokenizer,TFLongformerModel
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.initializers import RandomUniform

from numpy.random import seed
import random as python_random
import os
import sys
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

np.random.seed(1)
python_random.seed(1)
tf.random.set_seed(1)

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-09-16 02:09:34.345080: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-16 02:09:36.355759: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78949 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:31:00.0, compute capability: 8.0
2022-09-16 02:09:36.361209: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/t

In [ ]:
!cp "/content/drive/My Drive/labels_sc.txt" "./labels_sc.txt"
!cp "/content/drive/My Drive/labels_sc_279.txt" "./labels_sc_279.txt"

In [3]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2


In [4]:

def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    dense_0 = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    dropout_0= Dropout(0.5)(dense_0)
    pred = Dense(15, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout_0)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model   

In [5]:
total_accuracy=0
total_weighted_f1=0
total_micro_f1=0
total_weighted_precision=0
total_micro_precision=0
total_weighted_recall=0
total_micro_recall=0

In [7]:
with strategy.scope():

    sc = SupremeCourt()
    print(sc.info)
    sc.download()

    texts = []  # list of text samples
    labels_index = {}  # dictionary mapping label name to numeric id
    labels = []  # list of label ids

    issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
    print(issue_codes)
    issue_codes.sort()
    issue_codes = [str(ic) for ic in issue_codes]

    labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))
    print(labels_index)
    count=0

    for record in sc.records():

            count=count+1
            if record[1]['issue'] == None: # some cases have None as an issue
                labels.append(labels_index['-1'])
            else:
                labels.append(labels_index[record[1]['issue'][:-4]])

            new_sen=record[0].split("Footnotes")[0]
            new_new_sen=new_sen.split()
            if len(new_new_sen) >= 4096:
              new_new_sen=new_new_sen[0:4096]

            elif len(new_new_sen) < 4096:
              new_new_sen=new_new_sen[0:len(new_new_sen)]

            new_new_sen=' '.join(new_new_sen)

            texts.append(new_new_sen)

    len_list = [len(ele.split()) for ele in texts]

    #print(labels)
    print(len(labels))

    res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
    print("Average Length %s" % res) 
    print('Found %s texts.' % len(texts))
    print('Found %s labels.' % len(labels_index))
    temp_file = open("labels_sc.txt", "r")

    data = temp_file.read()
    label_list = data.split("\n")
    #print(label_list)
    label_list = label_list[0:-1]
    #print(label_list)
    label_list = [int(i) for i in label_list]
    summarized_data = pd.DataFrame(texts,
                   columns =['text'])
    summarized_data['label'] = label_list
    print(summarized_data)
    #gc.collect()
    #tf.keras.backend.clear_session()
    dbert_tokenizer = LongformerTokenizer.from_pretrained('saibo/legal-longformer-base-4096')
    dbert_model = TFLongformerModel.from_pretrained('saibo/legal-longformer-base-4096')
    max_len=4096
    sentences=summarized_data['text']
    labels=summarized_data['label']
    len(sentences),len(labels)
    model_0=create_model()
    input_ids=[]
    attention_masks=[]

    for sent in sentences:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids.append(dbert_inps['input_ids'])
      attention_masks.append(dbert_inps['attention_mask'])
    input_ids=np.asarray(input_ids)

    attention_masks=np.array(attention_masks)
    labels=np.array(labels)
    train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.1,random_state=42)
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    train_inp=train_inp[0:7576]
    train_mask=train_mask[0:7576]
    train_label=train_label[0:7576]

    val_inp=val_inp[0:840]
    val_label=val_label[0:840]
    val_mask=val_mask[0:840]
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    
    log_dir='dbert_model'

    model_save_path='LSM/legallongformer-0-15labels.h5'


    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
    model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
          

     
    

{'name': 'supreme_court', 'site_url': 'http://caselaw.findlaw.com/court/us-supreme-court', 'description': 'Collection of ~8.4k decisions issued by the U.S. Supreme Court between November 1946 and June 2016.'}
[-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
{'-1': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14}
8419
Average Length 2920.3854377004395
Found 8419 texts.
Found 15 labels.
                                                   text  label
0     [ Halliburton Oil Well Cementing Co. v. Walker...      8
1     Rehearing Denied Dec. 16, 1946. See . Mr.Claud...      1
2     Rehearing Denied Dec. 16, 1946 See . Appeal fr...      8
3     Mr. Walter J. Cummings, Jr., of Washington, D....      2
4     Mr.A. Devitt Vaneck, of Washington, D.C., for ...      8
...                                                 ...    ...
8414  Opinion reported: Ante, p. 88. DECREE 1. It is...     11
8415  In this dispute between

2022-09-16 02:11:08.573312: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Some layers from the model checkpoint at saibo/legal-longformer-base-4096 were not used when initializing TFLongformerModel: ['lm_head']
- This IS expected if you are initializing TFLongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFLongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFLongformerModel were initialized from the model checkpoint at saibo/legal-longformer-base-4096.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongf

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 tf_longformer_model (TFLongfor  TFLongformerBaseMod  148659456  ['input_1[0][0]',                
 merModel)                      elOutputWithPooling               'input_2[0][0]']                
                                (last_hidden_state=                                               
                                (None, 4096, 768),                                            

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(7577, 4096)
(7577, 4096)
(7577,)
(842, 4096)
(842, 4096)
(842,)
(7576, 4096)
(7576, 4096)
(7576,)
(840, 4096)
(840, 4096)
(840,)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job

In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)
  

Fri Sep 16 02:10:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:31:00.0 Off |                    0 |
| N/A   26C    P0    69W / 500W |  79534MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:CA:00.0 Off |                    0 |
| N/A   

In [20]:
gc.collect()
tf.keras.backend.clear_session()
history=model_0.fit([train_inp,train_mask],train_label,batch_size=8,epochs=5,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

2022-09-14 00:59:40.659063: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_642346"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:32848"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 8
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset.

Epoch 1/5


/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


947/947 [==============================] - ETA: 0s - loss: 6.7976 - accuracy: 0.6147

2022-09-14 01:16:22.744033: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_781486"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:32873"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 8
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset.

947/947 [==============================] - 1064s 970ms/step - loss: 6.7976 - accuracy: 0.6147 - val_loss: 5.7496 - val_accuracy: 0.7452
Epoch 2/5
947/947 [==============================] - 893s 943ms/step - loss: 5.2209 - accuracy: 0.7785 - val_loss: 4.8524 - val_accuracy: 0.7476
Epoch 3/5
947/947 [==============================] - 893s 943ms/step - loss: 4.2171 - accuracy: 0.8394 - val_loss: 4.0660 - val_accuracy: 0.7762
Epoch 4/5
947/947 [==============================] - 892s 942ms/step - loss: 3.3730 - accuracy: 0.8836 - val_loss: 3.5361 - val_accuracy: 0.7560
Epoch 5/5
947/947 [==============================] - 899s 949ms/step - loss: 2.6938 - accuracy: 0.9159 - val_loss: 3.1467 - val_accuracy: 0.7619


In [8]:
pred_labels=[]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_saved= create_model()
model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
model_saved.load_weights('LSM/legallongformer-0-15labels.h5')


print(val_inp.shape)
print(val_mask.shape)
print(val_label.shape)
eval_loss, eval_acc=model_saved.evaluate([val_inp,val_mask],val_label,batch_size=8)
print(eval_acc)


for i in range(0,len(val_inp)):
    pred=model_saved.predict([val_inp[i].reshape(1,4096),val_mask[i].reshape(1,4096)],batch_size=1)
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
accuracy=accuracy_score(val_label, pred_labels)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(val_label,pred_labels, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(val_label,pred_labels, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(val_label, pred_labels, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(val_label, pred_labels, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(val_label, pred_labels, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(val_label, pred_labels, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 tf_longformer_model (TFLongfor  TFLongformerBaseMod  148659456  ['input_3[0][0]',                
 merModel)                      elOutputWithPooling               'input_4[0][0]']                
                                (last_hidden_state=                                               
                                (None, 4096, 768),                                          

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


105/105 [==============================] - 83s 649ms/step - loss: 4.0660 - accuracy: 0.7762
0.776190459728241
Accuracy: 0.7761904761904762
Weighted F1: 0.77158880092525
Micro F1: 0.7761904761904762
Weighted Precision: 0.7754148318077306
Micro Precision: 0.7761904761904762
Weighted Recall: 0.7761904761904762
Micro Recall: 0.7761904761904762


/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
with strategy.scope():
    sc = SupremeCourt()
    print(sc.info)
    sc.download()

    texts = []  # list of text samples
    labels_index = {}  # dictionary mapping label name to numeric id
    labels = []  # list of label ids

    issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
    print(issue_codes)
    issue_codes.sort()
    issue_codes = [str(ic) for ic in issue_codes]

    labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))
    print(labels_index)
    count=0

    for record in sc.records():

            count=count+1
            if record[1]['issue'] == None: # some cases have None as an issue
                labels.append(labels_index['-1'])
            else:
                labels.append(labels_index[record[1]['issue'][:-4]])

            new_sen=record[0].split("Footnotes")[0]
            new_new_sen=new_sen.split()
            if len(new_new_sen) >= 4096:
              new_new_sen=new_new_sen[0:4096]

            elif len(new_new_sen) < 4096:
              new_new_sen=new_new_sen[0:len(new_new_sen)]

            new_new_sen=' '.join(new_new_sen)

            texts.append(new_new_sen)

    len_list = [len(ele.split()) for ele in texts]

    #print(labels)
    print(len(labels))

    res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
    print("Average Length %s" % res) 
    print('Found %s texts.' % len(texts))
    print('Found %s labels.' % len(labels_index))
    temp_file = open("labels_sc.txt", "r")

    data = temp_file.read()
    label_list = data.split("\n")
    #print(label_list)
    label_list = label_list[0:-1]
    #print(label_list)
    label_list = [int(i) for i in label_list]
    summarized_data = pd.DataFrame(texts,
                   columns =['text'])
    summarized_data['label'] = label_list
    print(summarized_data)
    #gc.collect()
    #tf.keras.backend.clear_session()
    dbert_tokenizer = LongformerTokenizer.from_pretrained('saibo/legal-longformer-base-4096')
    dbert_model = TFLongformerModel.from_pretrained('saibo/legal-longformer-base-4096')
    max_len=4096
    sentences=summarized_data['text']
    labels=summarized_data['label']
    len(sentences),len(labels)
    model_0=create_model()
    input_ids=[]
    attention_masks=[]

    for sent in sentences:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids.append(dbert_inps['input_ids'])
      attention_masks.append(dbert_inps['attention_mask'])
    input_ids=np.asarray(input_ids)

    attention_masks=np.array(attention_masks)
    labels=np.array(labels)
    train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.1,random_state=42)
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    train_inp=train_inp[0:7576]
    train_mask=train_mask[0:7576]
    train_label=train_label[0:7576]

    val_inp=val_inp[0:840]
    val_label=val_label[0:840]
    val_mask=val_mask[0:840]
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    log_dir='dbert_model'

    model_save_path='LSM/legallongformer-1-15labels.h5'

    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
    model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])

{'name': 'supreme_court', 'site_url': 'http://caselaw.findlaw.com/court/us-supreme-court', 'description': 'Collection of ~8.4k decisions issued by the U.S. Supreme Court between November 1946 and June 2016.'}
[-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
{'-1': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14}
8419
Average Length 2920.3854377004395
Found 8419 texts.
Found 15 labels.
                                                   text  label
0     [ Halliburton Oil Well Cementing Co. v. Walker...      8
1     Rehearing Denied Dec. 16, 1946. See . Mr.Claud...      1
2     Rehearing Denied Dec. 16, 1946 See . Appeal fr...      8
3     Mr. Walter J. Cummings, Jr., of Washington, D....      2
4     Mr.A. Devitt Vaneck, of Washington, D.C., for ...      8
...                                                 ...    ...
8414  Opinion reported: Ante, p. 88. DECREE 1. It is...     11
8415  In this dispute between

Some layers from the model checkpoint at saibo/legal-longformer-base-4096 were not used when initializing TFLongformerModel: ['lm_head']
- This IS expected if you are initializing TFLongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFLongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFLongformerModel were initialized from the model checkpoint at saibo/legal-longformer-base-4096.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 tf_longformer_model (TFLongfor  TFLongformerBaseMod  148659456  ['input_3[0][0]',                
 merModel)                      elOutputWithPooling               'input_4[0][0]']                
                                (last_hidden_state=                                               
                                (None, 4096, 768),                                          

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(7577, 4096)
(7577, 4096)
(7577,)
(842, 4096)
(842, 4096)
(842,)
(7576, 4096)
(7576, 4096)
(7576,)
(840, 4096)
(840, 4096)
(840,)


In [23]:
gc.collect()
tf.keras.backend.clear_session()
history=model_0.fit([train_inp,train_mask],train_label,batch_size=8,epochs=5,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

2022-09-14 02:23:31.998757: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_989651"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:42183"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 8
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset.

Epoch 1/5


/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


947/947 [==============================] - ETA: 0s - loss: 6.7556 - accuracy: 0.6412

2022-09-14 02:40:25.065045: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_1128791"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:42208"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 8
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset

947/947 [==============================] - 1075s 984ms/step - loss: 6.7556 - accuracy: 0.6412 - val_loss: 5.7645 - val_accuracy: 0.7369
Epoch 2/5
947/947 [==============================] - 908s 959ms/step - loss: 5.1765 - accuracy: 0.7810 - val_loss: 4.7889 - val_accuracy: 0.7667
Epoch 3/5
947/947 [==============================] - 906s 957ms/step - loss: 4.1450 - accuracy: 0.8449 - val_loss: 4.0304 - val_accuracy: 0.7726
Epoch 4/5
947/947 [==============================] - 905s 956ms/step - loss: 3.2874 - accuracy: 0.8903 - val_loss: 3.4558 - val_accuracy: 0.7631
Epoch 5/5
947/947 [==============================] - 907s 957ms/step - loss: 2.6130 - accuracy: 0.9187 - val_loss: 3.0057 - val_accuracy: 0.7679


In [9]:
pred_labels=[]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_saved= create_model()
model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
model_saved.load_weights('LSM/legallongformer-1-15labels.h5')

for i in range(0,len(val_inp)):
    pred=model_saved.predict([val_inp[i].reshape(1,4096),val_mask[i].reshape(1,4096)],batch_size=8)
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
accuracy=accuracy_score(val_label, pred_labels)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(val_label,pred_labels, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(val_label,pred_labels, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(val_label, pred_labels, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(val_label, pred_labels, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(val_label, pred_labels, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(val_label, pred_labels, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 tf_longformer_model (TFLongfor  TFLongformerBaseMod  148659456  ['input_5[0][0]',                
 merModel)                      elOutputWithPooling               'input_6[0][0]']                
                                (last_hidden_state=                                               
                                (None, 4096, 768),                                          

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
with strategy.scope():
    sc = SupremeCourt()
    print(sc.info)
    sc.download()

    texts = []  # list of text samples
    labels_index = {}  # dictionary mapping label name to numeric id
    labels = []  # list of label ids

    issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
    print(issue_codes)
    issue_codes.sort()
    issue_codes = [str(ic) for ic in issue_codes]

    labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))
    print(labels_index)
    count=0

    for record in sc.records():

            count=count+1
            if record[1]['issue'] == None: # some cases have None as an issue
                labels.append(labels_index['-1'])
            else:
                labels.append(labels_index[record[1]['issue'][:-4]])

            new_sen=record[0].split("Footnotes")[0]
            new_new_sen=new_sen.split()
            if len(new_new_sen) >= 4096:
              new_new_sen=new_new_sen[0:4096]

            elif len(new_new_sen) < 4096:
              new_new_sen=new_new_sen[0:len(new_new_sen)]

            new_new_sen=' '.join(new_new_sen)

            texts.append(new_new_sen)

    len_list = [len(ele.split()) for ele in texts]

    #print(labels)
    print(len(labels))

    res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
    print("Average Length %s" % res) 
    print('Found %s texts.' % len(texts))
    print('Found %s labels.' % len(labels_index))
    temp_file = open("labels_sc.txt", "r")

    data = temp_file.read()
    label_list = data.split("\n")
    #print(label_list)
    label_list = label_list[0:-1]
    #print(label_list)
    label_list = [int(i) for i in label_list]
    summarized_data = pd.DataFrame(texts,
                   columns =['text'])
    summarized_data['label'] = label_list
    print(summarized_data)
    #gc.collect()
    #tf.keras.backend.clear_session()
    dbert_tokenizer = LongformerTokenizer.from_pretrained('saibo/legal-longformer-base-4096')
    dbert_model = TFLongformerModel.from_pretrained('saibo/legal-longformer-base-4096')
    max_len=4096
    sentences=summarized_data['text']
    labels=summarized_data['label']
    len(sentences),len(labels)
    model_0=create_model()
    input_ids=[]
    attention_masks=[]

    for sent in sentences:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids.append(dbert_inps['input_ids'])
      attention_masks.append(dbert_inps['attention_mask'])
    input_ids=np.asarray(input_ids)

    attention_masks=np.array(attention_masks)
    labels=np.array(labels)
    train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.1,random_state=42)
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    train_inp=train_inp[0:7576]
    train_mask=train_mask[0:7576]
    train_label=train_label[0:7576]

    val_inp=val_inp[0:840]
    val_label=val_label[0:840]
    val_mask=val_mask[0:840]
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    log_dir='dbert_model'

    model_save_path='LSM/legallongformer-2-15labels.h5'

    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
    model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])

{'name': 'supreme_court', 'site_url': 'http://caselaw.findlaw.com/court/us-supreme-court', 'description': 'Collection of ~8.4k decisions issued by the U.S. Supreme Court between November 1946 and June 2016.'}
[-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
{'-1': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14}
8419
Average Length 2920.3854377004395
Found 8419 texts.
Found 15 labels.
                                                   text  label
0     [ Halliburton Oil Well Cementing Co. v. Walker...      8
1     Rehearing Denied Dec. 16, 1946. See . Mr.Claud...      1
2     Rehearing Denied Dec. 16, 1946 See . Appeal fr...      8
3     Mr. Walter J. Cummings, Jr., of Washington, D....      2
4     Mr.A. Devitt Vaneck, of Washington, D.C., for ...      8
...                                                 ...    ...
8414  Opinion reported: Ante, p. 88. DECREE 1. It is...     11
8415  In this dispute between

2022-09-14 11:51:26.499913: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Some layers from the model checkpoint at saibo/legal-longformer-base-4096 were not used when initializing TFLongformerModel: ['lm_head']
- This IS expected if you are initializing TFLongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFLongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFLongformerModel were initialized from the model checkpoint at saibo/legal-longformer-base-4096.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongf

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 tf_longformer_model (TFLongfor  TFLongformerBaseMod  148659456  ['input_1[0][0]',                
 merModel)                      elOutputWithPooling               'input_2[0][0]']                
                                (last_hidden_state=                                               
                                (None, 4096, 768),                                            

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(7577, 4096)
(7577, 4096)
(7577,)
(842, 4096)
(842, 4096)
(842,)
(7576, 4096)
(7576, 4096)
(7576,)
(840, 4096)
(840, 4096)
(840,)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job

In [7]:
gc.collect()
tf.keras.backend.clear_session()
history=model_0.fit([train_inp,train_mask],train_label,batch_size=8,epochs=5,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

2022-09-14 11:53:49.799720: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_50122"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\020FlatMapDataset:4"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 8
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You 

Epoch 1/5


/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


INFO:tensorflow:batch_all_reduce: 270 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 270 all-reduces with algorithm = nccl,

2022-09-14 12:10:09.700022: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_189266"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\021FlatMapDataset:29"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 8
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. Yo

947/947 [==============================] - 1040s 944ms/step - loss: 6.7856 - accuracy: 0.6172 - val_loss: 5.7542 - val_accuracy: 0.7429
Epoch 2/5
947/947 [==============================] - 869s 918ms/step - loss: 5.2215 - accuracy: 0.7860 - val_loss: 4.8713 - val_accuracy: 0.7524
Epoch 3/5
947/947 [==============================] - 869s 918ms/step - loss: 4.2033 - accuracy: 0.8445 - val_loss: 4.0300 - val_accuracy: 0.7667
Epoch 4/5
947/947 [==============================] - 868s 917ms/step - loss: 3.3622 - accuracy: 0.8873 - val_loss: 3.4671 - val_accuracy: 0.7702
Epoch 5/5
947/947 [==============================] - 869s 918ms/step - loss: 2.6731 - accuracy: 0.9209 - val_loss: 3.0595 - val_accuracy: 0.7702


In [10]:
pred_labels=[]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_saved= create_model()
model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
model_saved.load_weights('LSM/legallongformer-2-15labels.h5')

for i in range(0,len(val_inp)):
    pred=model_saved.predict([val_inp[i].reshape(1,4096),val_mask[i].reshape(1,4096)],batch_size=8)
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
accuracy=accuracy_score(val_label, pred_labels)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(val_label,pred_labels, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(val_label,pred_labels, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(val_label, pred_labels, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(val_label, pred_labels, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(val_label, pred_labels, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(val_label, pred_labels, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 tf_longformer_model (TFLongfor  TFLongformerBaseMod  148659456  ['input_7[0][0]',                
 merModel)                      elOutputWithPooling               'input_8[0][0]']                
                                (last_hidden_state=                                               
                                (None, 4096, 768),                                          

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
with strategy.scope():
    sc = SupremeCourt()
    print(sc.info)
    sc.download()

    texts = []  # list of text samples
    labels_index = {}  # dictionary mapping label name to numeric id
    labels = []  # list of label ids

    issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
    print(issue_codes)
    issue_codes.sort()
    issue_codes = [str(ic) for ic in issue_codes]

    labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))
    print(labels_index)
    count=0

    for record in sc.records():

            count=count+1
            if record[1]['issue'] == None: # some cases have None as an issue
                labels.append(labels_index['-1'])
            else:
                labels.append(labels_index[record[1]['issue'][:-4]])

            new_sen=record[0].split("Footnotes")[0]
            new_new_sen=new_sen.split()
            if len(new_new_sen) >= 4096:
              new_new_sen=new_new_sen[0:4096]

            elif len(new_new_sen) < 4096:
              new_new_sen=new_new_sen[0:len(new_new_sen)]

            new_new_sen=' '.join(new_new_sen)

            texts.append(new_new_sen)

    len_list = [len(ele.split()) for ele in texts]

    #print(labels)
    print(len(labels))

    res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
    print("Average Length %s" % res) 
    print('Found %s texts.' % len(texts))
    print('Found %s labels.' % len(labels_index))
    temp_file = open("labels_sc.txt", "r")

    data = temp_file.read()
    label_list = data.split("\n")
    #print(label_list)
    label_list = label_list[0:-1]
    #print(label_list)
    label_list = [int(i) for i in label_list]
    summarized_data = pd.DataFrame(texts,
                   columns =['text'])
    summarized_data['label'] = label_list
    print(summarized_data)
    #gc.collect()
    #tf.keras.backend.clear_session()
    dbert_tokenizer = LongformerTokenizer.from_pretrained('saibo/legal-longformer-base-4096')
    dbert_model = TFLongformerModel.from_pretrained('saibo/legal-longformer-base-4096')
    max_len=4096
    sentences=summarized_data['text']
    labels=summarized_data['label']
    len(sentences),len(labels)
    model_0=create_model()
    input_ids=[]
    attention_masks=[]

    for sent in sentences:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids.append(dbert_inps['input_ids'])
      attention_masks.append(dbert_inps['attention_mask'])
    input_ids=np.asarray(input_ids)

    attention_masks=np.array(attention_masks)
    labels=np.array(labels)
    train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.1,random_state=42)
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    train_inp=train_inp[0:7576]
    train_mask=train_mask[0:7576]
    train_label=train_label[0:7576]

    val_inp=val_inp[0:840]
    val_label=val_label[0:840]
    val_mask=val_mask[0:840]
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    log_dir='dbert_model'

    model_save_path='LSM/legallongformer-3-15labels.h5'

    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
    model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])

{'name': 'supreme_court', 'site_url': 'http://caselaw.findlaw.com/court/us-supreme-court', 'description': 'Collection of ~8.4k decisions issued by the U.S. Supreme Court between November 1946 and June 2016.'}
[-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
{'-1': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14}
8419
Average Length 2920.3854377004395
Found 8419 texts.
Found 15 labels.
                                                   text  label
0     [ Halliburton Oil Well Cementing Co. v. Walker...      8
1     Rehearing Denied Dec. 16, 1946. See . Mr.Claud...      1
2     Rehearing Denied Dec. 16, 1946 See . Appeal fr...      8
3     Mr. Walter J. Cummings, Jr., of Washington, D....      2
4     Mr.A. Devitt Vaneck, of Washington, D.C., for ...      8
...                                                 ...    ...
8414  Opinion reported: Ante, p. 88. DECREE 1. It is...     11
8415  In this dispute between

Some layers from the model checkpoint at saibo/legal-longformer-base-4096 were not used when initializing TFLongformerModel: ['lm_head']
- This IS expected if you are initializing TFLongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFLongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFLongformerModel were initialized from the model checkpoint at saibo/legal-longformer-base-4096.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 tf_longformer_model (TFLongfor  TFLongformerBaseMod  148659456  ['input_3[0][0]',                
 merModel)                      elOutputWithPooling               'input_4[0][0]']                
                                (last_hidden_state=                                               
                                (None, 4096, 768),                                          

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(7577, 4096)
(7577, 4096)
(7577,)
(842, 4096)
(842, 4096)
(842,)
(7576, 4096)
(7576, 4096)
(7576,)
(840, 4096)
(840, 4096)
(840,)


In [10]:
gc.collect()
tf.keras.backend.clear_session()
history=model_0.fit([train_inp,train_mask],train_label,batch_size=8,epochs=5,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

2022-09-14 13:14:20.794299: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_393683"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:11008"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 8
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset.

Epoch 1/5


/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


947/947 [==============================] - ETA: 0s - loss: 6.7597 - accuracy: 0.6408

2022-09-14 13:30:47.175219: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_532823"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:11033"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 8
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset.

947/947 [==============================] - 1047s 955ms/step - loss: 6.7597 - accuracy: 0.6408 - val_loss: 5.7338 - val_accuracy: 0.7476
Epoch 2/5
947/947 [==============================] - 876s 925ms/step - loss: 5.1899 - accuracy: 0.7789 - val_loss: 4.7964 - val_accuracy: 0.7643
Epoch 3/5
947/947 [==============================] - 876s 925ms/step - loss: 4.1445 - accuracy: 0.8454 - val_loss: 4.0534 - val_accuracy: 0.7738
Epoch 4/5
947/947 [==============================] - 875s 924ms/step - loss: 3.2942 - accuracy: 0.8904 - val_loss: 3.4981 - val_accuracy: 0.7571
Epoch 5/5
947/947 [==============================] - 876s 925ms/step - loss: 2.6170 - accuracy: 0.9175 - val_loss: 2.9994 - val_accuracy: 0.7810


In [11]:
pred_labels=[]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_saved= create_model()
model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
model_saved.load_weights('LSM/legallongformer-3-15labels.h5')

for i in range(0,len(val_inp)):
    pred=model_saved.predict([val_inp[i].reshape(1,4096),val_mask[i].reshape(1,4096)],batch_size=8)
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
accuracy=accuracy_score(val_label, pred_labels)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(val_label,pred_labels, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(val_label,pred_labels, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(val_label, pred_labels, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(val_label, pred_labels, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(val_label, pred_labels, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(val_label, pred_labels, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 input_10 (InputLayer)          [(None, 4096)]       0           []                               
                                                                                                  
 tf_longformer_model (TFLongfor  TFLongformerBaseMod  148659456  ['input_9[0][0]',                
 merModel)                      elOutputWithPooling               'input_10[0][0]']               
                                (last_hidden_state=                                               
                                (None, 4096, 768),                                          

In [12]:
with strategy.scope():
    sc = SupremeCourt()
    print(sc.info)
    sc.download()

    texts = []  # list of text samples
    labels_index = {}  # dictionary mapping label name to numeric id
    labels = []  # list of label ids

    issue_codes = list(sc.issue_area_codes.keys()) # 15 labels
    print(issue_codes)
    issue_codes.sort()
    issue_codes = [str(ic) for ic in issue_codes]

    labels_index = dict(zip(issue_codes, np.arange(len(issue_codes))))
    print(labels_index)
    count=0

    for record in sc.records():

            count=count+1
            if record[1]['issue'] == None: # some cases have None as an issue
                labels.append(labels_index['-1'])
            else:
                labels.append(labels_index[record[1]['issue'][:-4]])

            new_sen=record[0].split("Footnotes")[0]
            new_new_sen=new_sen.split()
            if len(new_new_sen) >= 4096:
              new_new_sen=new_new_sen[0:4096]

            elif len(new_new_sen) < 4096:
              new_new_sen=new_new_sen[0:len(new_new_sen)]

            new_new_sen=' '.join(new_new_sen)

            texts.append(new_new_sen)

    len_list = [len(ele.split()) for ele in texts]

    #print(labels)
    print(len(labels))

    res = 0 if len(len_list) == 0 else (float(sum(len_list)) / len(len_list))
    print("Average Length %s" % res) 
    print('Found %s texts.' % len(texts))
    print('Found %s labels.' % len(labels_index))
    temp_file = open("labels_sc.txt", "r")

    data = temp_file.read()
    label_list = data.split("\n")
    #print(label_list)
    label_list = label_list[0:-1]
    #print(label_list)
    label_list = [int(i) for i in label_list]
    summarized_data = pd.DataFrame(texts,
                   columns =['text'])
    summarized_data['label'] = label_list
    print(summarized_data)
    #gc.collect()
    #tf.keras.backend.clear_session()
    dbert_tokenizer = LongformerTokenizer.from_pretrained('saibo/legal-longformer-base-4096')
    dbert_model = TFLongformerModel.from_pretrained('saibo/legal-longformer-base-4096')
    max_len=4096
    sentences=summarized_data['text']
    labels=summarized_data['label']
    len(sentences),len(labels)
    model_0=create_model()
    input_ids=[]
    attention_masks=[]

    for sent in sentences:
      dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
      input_ids.append(dbert_inps['input_ids'])
      attention_masks.append(dbert_inps['attention_mask'])
    input_ids=np.asarray(input_ids)

    attention_masks=np.array(attention_masks)
    labels=np.array(labels)
    train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.1,random_state=42)
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    train_inp=train_inp[0:7576]
    train_mask=train_mask[0:7576]
    train_label=train_label[0:7576]

    val_inp=val_inp[0:840]
    val_label=val_label[0:840]
    val_mask=val_mask[0:840]
    print(train_inp.shape)
    print(train_mask.shape)
    print(train_label.shape)
    print(val_inp.shape)
    print(val_mask.shape)
    print(val_label.shape)
    log_dir='dbert_model'

    model_save_path='LSM/legallongformer-4-15labels.h5'

    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,monitor='val_accuracy',mode='max',save_best_only=True,save_weights_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
    model_0.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])

{'name': 'supreme_court', 'site_url': 'http://caselaw.findlaw.com/court/us-supreme-court', 'description': 'Collection of ~8.4k decisions issued by the U.S. Supreme Court between November 1946 and June 2016.'}
[-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
{'-1': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14}
8419
Average Length 2920.3854377004395
Found 8419 texts.
Found 15 labels.
                                                   text  label
0     [ Halliburton Oil Well Cementing Co. v. Walker...      8
1     Rehearing Denied Dec. 16, 1946. See . Mr.Claud...      1
2     Rehearing Denied Dec. 16, 1946 See . Appeal fr...      8
3     Mr. Walter J. Cummings, Jr., of Washington, D....      2
4     Mr.A. Devitt Vaneck, of Washington, D.C., for ...      8
...                                                 ...    ...
8414  Opinion reported: Ante, p. 88. DECREE 1. It is...     11
8415  In this dispute between

Some layers from the model checkpoint at saibo/legal-longformer-base-4096 were not used when initializing TFLongformerModel: ['lm_head']
- This IS expected if you are initializing TFLongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFLongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFLongformerModel were initialized from the model checkpoint at saibo/legal-longformer-base-4096.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 tf_longformer_model (TFLongfor  TFLongformerBaseMod  148659456  ['input_3[0][0]',                
 merModel)                      elOutputWithPooling               'input_4[0][0]']                
                                (last_hidden_state=                                               
                                (None, 4096, 768),                                          

/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


(7577, 4096)
(7577, 4096)
(7577,)
(842, 4096)
(842, 4096)
(842,)
(7576, 4096)
(7576, 4096)
(7576,)
(840, 4096)
(840, 4096)
(840,)


In [13]:
gc.collect()
tf.keras.backend.clear_session()
history=model_0.fit([train_inp,train_mask],train_label,batch_size=8,epochs=5,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

2022-09-14 14:35:33.099568: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_737240"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:22012"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 8
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset.

Epoch 1/5


/ext3/miniconda3/envs/wol_env/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


947/947 [==============================] - ETA: 0s - loss: 6.9106 - accuracy: 0.6364

2022-09-14 14:51:52.563123: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_876380"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024FlatMapDataset:22037"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 8
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset.

947/947 [==============================] - 1039s 947ms/step - loss: 6.9106 - accuracy: 0.6364 - val_loss: 5.9838 - val_accuracy: 0.7369
Epoch 2/5
947/947 [==============================] - 870s 918ms/step - loss: 5.3999 - accuracy: 0.7839 - val_loss: 5.0074 - val_accuracy: 0.7679
Epoch 3/5
947/947 [==============================] - 870s 918ms/step - loss: 4.3638 - accuracy: 0.8355 - val_loss: 4.2023 - val_accuracy: 0.7750
Epoch 4/5
947/947 [==============================] - 869s 918ms/step - loss: 3.4886 - accuracy: 0.8850 - val_loss: 3.5737 - val_accuracy: 0.7786
Epoch 5/5
947/947 [==============================] - 870s 918ms/step - loss: 2.7679 - accuracy: 0.9204 - val_loss: 3.1829 - val_accuracy: 0.7643


In [ ]:
pred_labels=[]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model_saved= create_model()
model_saved.compile(loss=loss,optimizer=optimizer, metrics=[accuracy])
model_saved.load_weights('LSM/legallongformer-4-15labels.h5')

for i in range(0,len(val_inp)):
    pred=model_saved.predict([val_inp[i].reshape(1,4096),val_mask[i].reshape(1,4096)],batch_size=8)
    pred_label = pred.argmax(axis=1)
    pred_labels.append(pred_label)
accuracy=accuracy_score(val_label, pred_labels)
print("Accuracy: "+str(accuracy))
total_accuracy=total_accuracy+accuracy

weighted_f1=f1_score(val_label,pred_labels, average='weighted')
print("Weighted F1: "+ str(weighted_f1))
total_weighted_f1=total_weighted_f1+weighted_f1
micro_f1=f1_score(val_label,pred_labels, average='micro')
print("Micro F1: "+ str(micro_f1))
total_micro_f1=total_micro_f1+micro_f1

weighted_precision=precision_score(val_label, pred_labels, average='weighted')
print("Weighted Precision: " + str(weighted_precision))
total_weighted_precision=total_weighted_precision+weighted_precision
micro_precision=precision_score(val_label, pred_labels, average='micro')
print("Micro Precision: " + str(micro_precision))
total_micro_precision=total_micro_precision+micro_precision

weighted_recall=recall_score(val_label, pred_labels, average='weighted')
print("Weighted Recall: " + str(weighted_recall))
total_weighted_recall=total_weighted_recall+weighted_recall
micro_recall=recall_score(val_label, pred_labels, average='micro')
print("Micro Recall: " + str(micro_recall))
total_micro_recall=total_micro_recall+micro_recall

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 4096)]       0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, 4096)]       0           []                               
                                                                                                  
 tf_longformer_model (TFLongfor  TFLongformerBaseMod  148659456  ['input_11[0][0]',               
 merModel)                      elOutputWithPooling               'input_12[0][0]']               
                                (last_hidden_state=                                               
                                (None, 4096, 768),                                          

In [ ]:
print("Average Accuracy: "+str(total_accuracy/5))
print("Average Weighted F1: "+str(total_weighted_f1/5))
print("Average Micro F1: "+str(total_micro_f1/5))
print("Average Weighted Precision: "+str(total_weighted_precision/5))
print("Average Micro Precision: "+str(total_micro_precision/5))
print("Average Weighted Recall: "+str(total_weighted_recall/5))
print("Average Micro Recall: "+str(total_micro_recall/5))